Задача:
Использовать модель и датасет из предыдущих домашних заданий или промежуточной аттестации и построить полноценный Pipeline для обработки и подготовки данных к моделированию.

Цель:
Автоматизировать процесс подготовки данных для улучшения качества моделей и упрощения этапов предобработки.

Рекомендуемые шаги:
1. Загрузка и анализ данных - определите типы данных и распределение признаков.

2. Обработка пропущенных данных - используйте SimpleImputer или IterativeImputer для заполнения пропусков.

3. Кодирование категориальных данных - примените OneHotEncoder или TargetEncoder.

4. Масштабирование данных - используйте StandardScaler или RobustScaler.

5. Построение модели - примените RandomForest или LightGBM как пример.

Инструменты:
- sklearn.pipeline.Pipeline
- GridSearchCV для подбора гиперпараметров

### В следующем блоке содержится код, отрабатывающий процесс от загрузки датасета, до обучения модели и вывода результатов. Без проверок и с меньшим количеством комментариев.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import missingno as msno
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report

# Загружаем датасет Titanic
titanic = pd.read_csv('https://raw.githubusercontent.com/rogovich/Data/master/data/titanic/train.csv')

# Разделяем данные на обучающую и тестовую выборки
X = titanic.drop(['Survived', 'PassengerId', 'Ticket', 'Cabin'], axis = 1)
y = titanic['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Смотрим распределение пропусков по признакам
print(msno.bar(titanic))

# Эта визуализация, даёт нам более интуитивное представление о том, где отсутствуют значения
msno.matrix(titanic)
plt.show()

# Выведем корреляционную матрицу между признаками
corr_data = titanic.select_dtypes(include=['float64', 'int64']).corr()  # Выбираем столбцы на основе типа данных

# Задаём размер матрицы
plt.figure(figsize = (12,8))

# Поместим созданную выше корреляционную матрицу в функцию sns.heatmap()
sns.heatmap(corr_data, annot = True, fmt='.2g', annot_kws={'size':str(7)}, linewidths = .5, cmap= 'coolwarm')

# Объединяем признаки с наибольшей корреляцией (Parch и SibSp) в новые признаки
for dataset in [X_train, X_test]:
    dataset['family_size'] = dataset['Parch'] + dataset['SibSp']  # Общая численность круга семьи на борту
    dataset.drop(['Parch', 'SibSp'], axis = 1, inplace = True)
    dataset['is_alone'] = 1
    dataset['is_alone'] = dataset['family_size'].apply(lambda x: 1 if x >= 1 else 0)  # 0 - был один, 1 - с семьёй

# Извлекаем титулы пассажиров и сохраняем их в новый признак, называемый title
for dataset in [X_train, X_test]:
  dataset['title'] =  dataset['Name'].str.split(", ", expand = True)[1].str.split(".", expand = True)[0]
  dataset.drop(["Name"], axis = 1, inplace = True)

# Смотрим на количество редких титулов и группируем их
X_comb = pd.concat([X_train, X_test])
rare_titles = (X_comb['title'].value_counts() < 10)

for dataset in [X_train, X_test]:
    dataset['title'] = dataset['title'].replace({'Miss': 'Mrs'})  # Группировка Mrs и Miss в одну группу
    dataset['title'] = dataset['title'].apply(lambda x: 'rare' if rare_titles[x] else x)  # И всех редких в другую

# Разделяем признаки по типам данных
cat_cols = ['Embarked', 'Sex', 'Pclass', 'title', 'is_alone']
num_cols = ['Age', 'Fare', 'family_size']

# Функция для заполнения пропусков и кодирования категориальных данных
cat_transformer = Pipeline(steps = [
    ('imputer', SimpleImputer(strategy = 'most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown = 'ignore', sparse_output = False)),
    ('pca', PCA(n_components = 10))  # PCA для уменьшения размерности, после работы OneHotEncoder
])

# Функция для масштабирования числовых данных
num_transformer = Pipeline(steps = [
                          ('imputer', KNNImputer(n_neighbors = 5)),
                          ('scaler', RobustScaler())
])

# Строим трансформер для всех данных
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_cols),
        ('cat', cat_transformer, cat_cols)
    ])

# Строим модель
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', RandomForestClassifier())])

# Выполним поиск гиперпараметров с наилучшим результатом
num_transformer_dist = {'preprocessor__num__imputer__n_neighbors': [3, 5, 8],
                        'preprocessor__num__imputer__add_indicator': [True, False]}

cat_transformer_dist = {'preprocessor__cat__imputer__strategy': ['most_frequent', 'constant'],
                        'preprocessor__cat__imputer__add_indicator': [True, False],
                        'preprocessor__cat__pca__n_components': [3, 6, 10]}

random_forest_dist = {'classifier__n_estimators': [50, 100, 150],
                      'classifier__max_depth': list(range(2, 7)),
                      'classifier__bootstrap': [True, False]}

param_dist = {**num_transformer_dist, **cat_transformer_dist, **random_forest_dist}

# По заданию, нужно работать с GridSearchCV, но на моей машине на это ушло 21мин 45сек
# grid_search = GridSearchCV(clf,
#                             param_grid = param_dist,
#                             cv = 4)

# Работа RandomizedSearchCV с теми же вводными, на моей машине была выполнена за 1мин 18сек, без значимой потери в результатах
random_search = RandomizedSearchCV(clf,
                                   param_distributions = param_dist,
                                   n_iter = 100)

# Обучаем модель
random_search.fit(X_train, y_train)

# y_pred = grid_search.predict(X_test)
y_pred = random_search.predict(X_test)

# Выводим финальные результаты
print(classification_report(y_test, y_pred))

### В следующем блоке содержится тот же код что и выше, но со всеми комметариями и дополнениями, которые я делал в процессе выполнения задания.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import missingno as msno
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report

In [ ]:
# Загружаем датасет Titanic
titanic = pd.read_csv('https://raw.githubusercontent.com/rogovich/Data/master/data/titanic/train.csv')

# Разделяем данные на обучающую и тестовую выборки
X = titanic.drop(['Survived', 'PassengerId', 'Ticket', 'Cabin'], axis = 1)
y = titanic['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Смотрим первые строки датафрейма
X_train.head()

В датасете Titanic есть следующие столбцы:

Pclass — класс пассажира (1 — высший, 2 — средний, 3 — низший).

Name — имя.

Sex — пол.

Age — возраст.

SibSp — количество братьев, сестер, сводных братьев, сводных сестер, супругов на борту Титаника.

Parch — количество родителей, детей (в том числе приемных) на борту Титаника.

Ticket — номер билета.

Fare — плата за проезд.

Cabin — каюта.

Embarked — порт посадки (C — Шербур, Q — Квинстаун, S — Саутгемптон).

In [ ]:
# Смотрим распределение пропусков по признакам
msno.bar(titanic)

In [ ]:
# Эта визуализация, даёт нам более интуитивное представление о том, где отсутствуют значения
msno.matrix(titanic)
plt.show()

In [ ]:
# Смотрим типы данных
titanic.dtypes

In [ ]:
# Выведем корреляционную матрицу между признаками
corr_data = titanic.select_dtypes(include = ['float64', 'int64']).corr()  # Выбираем столбцы на основе типа данных

# Задаём размер матрицы
plt.figure(figsize = (12,8))

# Поместим созданную выше корреляционную матрицу в функцию sns.heatmap()
sns.heatmap(corr_data, annot = True, fmt = '.2g', annot_kws = {'size':str(7)}, linewidths = .5, cmap = 'coolwarm')

In [ ]:
# Объединяем признаки с наибольшей корреляцией (Parch и SibSp) в новые признаки
for dataset in [X_train, X_test]:
    dataset['family_size'] = dataset['Parch'] + dataset['SibSp']
    dataset.drop(['Parch', 'SibSp'], axis = 1, inplace = True)
    dataset['is_alone'] = 1
    dataset['is_alone'] = dataset['family_size'].apply(lambda x: 1 if x >= 1 else 0)

X_train.head(20)

In [ ]:
# Извлекаем титулы пассажиров и сохраняем их в новый признак, называемый title
for dataset in [X_train, X_test]:
  dataset['title'] =  dataset['Name'].str.split(", ", expand = True)[1].str.split(".", expand = True)[0]
  dataset.drop(["Name"], axis = 1, inplace = True)

X_train.head()

In [ ]:
# Смотрим на количество редких титулов. По факту, есть смысл их сгруппировать
X_comb = pd.concat([X_train, X_test])
rare_titles = (X_comb['title'].value_counts() < 10)
rare_titles

In [ ]:
for dataset in [X_train, X_test]:
    dataset['title'] = dataset['title'].replace({'Miss': 'Mrs'})  # Группировка Mrs и Miss в одну группу
    dataset['title'] = dataset['title'].apply(lambda x: 'rare' if rare_titles[x] else x)  # И всех редких в другую

In [ ]:
# Проверяем количество титулов
X_comb = pd.concat([X_train, X_test])
rare_titles = (X_comb['title'].value_counts() < 10)
rare_titles

In [ ]:
# Смотрим типы данных
X_train.dtypes

In [ ]:
# Разделяем признаки по типам данных
cat_cols = ['Embarked', 'Sex', 'Pclass', 'title', 'is_alone']
num_cols = ['Age', 'Fare', 'family_size']

# Функция для заполнения пропусков и кодирования категориальных данных
cat_transformer = Pipeline(steps = [
    ('imputer', SimpleImputer(strategy = 'most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown = 'ignore', sparse_output = False)),
    ('pca', PCA(n_components = 10))  # PCA для уменьшения размерности, после работы OneHotEncoder
])

# Функция для масштабирования числовых данных
num_transformer = Pipeline(steps = [
                          ('imputer', KNNImputer(n_neighbors = 5)),
                          ('scaler', RobustScaler())
])

# Строим трансформер для всех данных
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_cols),
        ('cat', cat_transformer, cat_cols)
    ])

In [ ]:
# Строим модель и смотрим результаты CV без настройки гиперпараметров
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', RandomForestClassifier())])

cross_val_score(clf, X_train, y_train, cv = 5, scoring = "accuracy").mean()  # Результат 0.8

In [ ]:
# Выполним поиск гиперпараметров с наилучшим результатом
num_transformer_dist = {'preprocessor__num__imputer__n_neighbors': [3, 5, 8],
                        'preprocessor__num__imputer__add_indicator': [True, False]}

cat_transformer_dist = {'preprocessor__cat__imputer__strategy': ['most_frequent', 'constant'],
                        'preprocessor__cat__imputer__add_indicator': [True, False],
                        'preprocessor__cat__pca__n_components': [3, 6, 10]}

random_forest_dist = {'classifier__n_estimators': [50, 100, 150],
                      'classifier__max_depth': list(range(2, 7)),
                      'classifier__bootstrap': [True, False]}

param_dist = {**num_transformer_dist, **cat_transformer_dist, **random_forest_dist}

# По заданию, нужно работать с GridSearchCV, но на моей машине на это ушло 21мин 45сек
# grid_search = GridSearchCV(clf,
#                             param_grid = param_dist,
#                             cv = 4)

# Работа RandomizedSearchCV на моей машине была выполнена за 1мин 18сек, без значимой потери в результатах
random_search = RandomizedSearchCV(clf,
                                   param_distributions = param_dist,
                                   n_iter = 100)

# Обучаем модель
# grid_search.fit(X_train, y_train)
random_search.fit(X_train, y_train)

In [ ]:
# Смотрим результат поиска лучших параметров
# grid_search.best_params_
random_search.best_params_

# Результаты работы GridSearchCV:
# {'classifier__bootstrap': True,
#  'classifier__max_depth': 6,
#  'classifier__n_estimators': 50,
#  'preprocessor__cat__imputer__add_indicator': True,
#  'preprocessor__cat__imputer__strategy': 'constant',
#  'preprocessor__cat__pca__n_components': 3,
#  'preprocessor__num__imputer__add_indicator': True,
#  'preprocessor__num__imputer__n_neighbors': 8}

# Результаты работы RandomizedSearchCV:
# {'preprocessor__num__imputer__n_neighbors': 5,
#  'preprocessor__num__imputer__add_indicator': True,
#  'preprocessor__cat__pca__n_components': 3,
#  'preprocessor__cat__imputer__strategy': 'most_frequent',
#  'preprocessor__cat__imputer__add_indicator': False,
#  'classifier__n_estimators': 100,
#  'classifier__max_depth': 6,
#  'classifier__bootstrap': False}

In [ ]:
# y_pred = grid_search.predict(X_test)
y_pred = random_search.predict(X_test)

y_pred[:5]

In [ ]:
# Выводим финальные результаты
print(classification_report(y_test, y_pred))

# Результаты с параметрами, определёнными после работы GridSearchCV
#               precision    recall  f1-score   support

#            0       0.82      0.90      0.86       105
#            1       0.84      0.72      0.77        74

#     accuracy                           0.83       179
#    macro avg       0.83      0.81      0.82       179
# weighted avg       0.83      0.83      0.82       179

# Результаты с параметрами, определёнными после работы RandomizedSearchCV
#               precision    recall  f1-score   support

#            0       0.81      0.90      0.86       105
#            1       0.84      0.70      0.76        74

#     accuracy                           0.82       179
#    macro avg       0.83      0.80      0.81       179
# weighted avg       0.82      0.82      0.82       179